In [26]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("weatherHistory.csv")

# Convert date column to datetime
df['Formatted Date'] = pd.to_datetime(df['Formatted Date'], utc=True)

# Extract useful columns
df['Month'] = df['Formatted Date'].dt.month
df['Day'] = df['Formatted Date'].dt.day
df['Year'] = df['Formatted Date'].dt.year

# Convert humidity to percentage
df['Humidity'] = df['Humidity'] * 100

# Month name to number
month_map = {
    'january': 1, 'february': 2, 'march': 3, 'april': 4,
    'may': 5, 'june': 6, 'july': 7, 'august': 8,
    'september': 9, 'october': 10, 'november': 11, 'december': 12
}

# Short form mapping
month_aliases = {
    'jan': 'january', 'feb': 'february', 'mar': 'march', 'apr': 'april',
    'jun': 'june', 'jul': 'july', 'aug': 'august', 'sep': 'september',
    'oct': 'october', 'nov': 'november', 'dec': 'december'
}
word_aliases = {
    'temp': 'temperature', 'humid': 'humidity', 'rains': 'rainy',
    'coldest': 'cold', 'hottest': 'hot'
}


In [39]:
# ✅ Month name to number mapping
month_to_number = {
    'january': 1, 'february': 2, 'march': 3, 'april': 4,
    'may': 5, 'june': 6, 'july': 7, 'august': 8,
    'september': 9, 'october': 10, 'november': 11, 'december': 12
}


In [32]:
def get_avg_temp(month):
    month_num = month_map.get(month)
    if month_num is None: return "❌ Invalid month."
    avg = df[df['Month'] == month_num]['Temperature (C)'].mean()
    return f"🌡️ The average temperature in {month.title()} is {avg:.2f} °C"

def get_avg_humidity(month):
    month_num = month_map.get(month)
    if month_num is None: return "❌ Invalid month."
    avg = df[df['Month'] == month_num]['Humidity'].mean()
    return f"💧 The average humidity in {month.title()} is {avg:.2f}%"

def get_rainy_days(month):
    month_num = month_map.get(month)
    if month_num is None: return "❌ Invalid month."
    rainy = df[(df['Month'] == month_num) & (df['Precip Type'] == 'rain')]
    rainy_days = rainy['Formatted Date'].dt.date.nunique()
    return f"🌧️ Number of rainy days in {month.title()}: {rainy_days} days"

def get_temp_extreme(user_input):
    avg_temps = df.groupby('Month')['Temperature (C)'].mean()
    if 'hot' in user_input:
        hottest_month = avg_temps.idxmax()
        for name, num in month_map.items():
            if num == hottest_month:
                return f"🔥 The hottest month is {name.title()} with avg temp {avg_temps.max():.2f} °C"
    elif 'cold' in user_input:
        coldest_month = avg_temps.idxmin()
        for name, num in month_map.items():
            if num == coldest_month:
                return f"❄️ The coldest month is {name.title()} with avg temp {avg_temps.min():.2f} °C"
    return "❓ Please ask for the hottest or coldest month."

def get_avg_wind(month):
    month_num = month_map.get(month)
    if month_num is None: return "❌ Invalid month."
    avg = df[df['Month'] == month_num]['Wind Speed (km/h)'].mean()
    return f"🌬️ Avg wind speed in {month.title()}: {avg:.2f} km/h"

def get_avg_visibility(month):
    month_num = month_map.get(month)
    if month_num is None: return "❌ Invalid month."
    avg = df[df['Month'] == month_num]['Visibility (km)'].mean()
    return f"🔎 Avg visibility in {month.title()}: {avg:.2f} km"

def get_avg_pressure(month):
    month_num = month_map.get(month)
    if month_num is None: return "❌ Invalid month."
    avg = df[df['Month'] == month_num]['Pressure (millibars)'].mean()
    return f"📉 Avg pressure in {month.title()}: {avg:.2f} millibar (mb)"

def get_monthly_summary(month):
    month_num = month_to_number.get(month.lower())
    if not month_num:
        return "❌ Invalid month."

    month_data = df[df['Month'] == month_num]

    if month_data.empty:
        return "⚠️ No data available for this month."

    avg_temp = month_data['Temperature (C)'].mean()
    avg_humidity = month_data['Humidity'].mean()
    rainy_days = month_data[month_data['Precip Type'] == 'rain']['Day'].nunique()
    snowy_days = month_data[month_data['Precip Type'] == 'snow']['Day'].nunique()
    avg_pressure = month_data['Pressure (millibars)'].mean()
    avg_visibility = month_data['Visibility (km)'].mean()
    avg_wind_speed = month_data['Wind Speed (km/h)'].mean()

    return f"""
📅 Weather Summary for {month.title()}:
🌡️ Average Temperature: {avg_temp:.2f} °C
💧 Average Humidity: {avg_humidity:.2f}
🌧️ Rainy Days: {rainy_days}
❄️ Snowy Days: {snowy_days}
📉 Avg Pressure: {avg_pressure:.2f} millibars
🌫️ Avg Visibility: {avg_visibility:.2f} km
🌬️ Avg Wind Speed: {avg_wind_speed:.2f} km/h
"""



In [36]:
def SmartWeatherBot():
    print("🤖 Hello! I’m SmartWeatherBot. Ask me about:")
    print("- average temperature (e.g. 'temperature in June')")
    print("- average humidity (e.g. 'humidity in August')")
    print("- number of rainy or snowy days (e.g. 'rainy days in April')")
    print("- hottest or coldest month (e.g. 'what's the coldest month')")
    print("- full summary (e.g. 'weather in November')")
    print("Type 'exit' to quit.\n")
    
    while True:
        user_input = input("🧑 You: ").lower()
        
        if user_input == 'exit':
            print("👋 Goodbye!")
            break

        elif "temperature" in user_input:
            for month in month_to_number.keys():
                if month in user_input:
                    print(get_avg_temp(month))
                    break
            else:
                print("⚠️ Please include a valid month.")
        
        elif "humidity" in user_input:
            for month in month_to_number.keys():
                if month in user_input:
                    print(get_avg_humidity(month))
                    break
            else:
                print("⚠️ Please include a valid month.")
        
        elif "rain" in user_input:
            for month in month_to_number.keys():
                if month in user_input:
                    print(get_rainy_days(month))
                    break
            else:
                print("⚠️ Please include a valid month.")

        elif "cold" in user_input or "hot" in user_input:
            print(get_temp_extreme(user_input))

        elif "weather in" in user_input or "summary" in user_input:
            for month in month_to_number.keys():
                if month in user_input:
                    print(get_monthly_summary(month))
                    break
            else:
                print("⚠️ Please include a valid month.")

        else:
            print("❓ I didn’t understand. Try asking about temperature, humidity, rain, snow, or summary.")


In [40]:
SmartWeatherBot()


🤖 Hello! I’m SmartWeatherBot. Ask me about:
- average temperature (e.g. 'temperature in June')
- average humidity (e.g. 'humidity in August')
- number of rainy or snowy days (e.g. 'rainy days in April')
- hottest or coldest month (e.g. 'what's the coldest month')
- full summary (e.g. 'weather in November')
Type 'exit' to quit.



🧑 You:  weather in june



📅 Weather Summary for June:
🌡️ Average Temperature: 20.73 °C
💧 Average Humidity: 68.64
🌧️ Rainy Days: 30
❄️ Snowy Days: 0
📉 Avg Pressure: 1002.02 millibars
🌫️ Avg Visibility: 11.99 km
🌬️ Avg Wind Speed: 9.63 km/h



🧑 You:  summary juky


⚠️ Please include a valid month.


...


📅 Weather Summary for July:
🌡️ Average Temperature: 22.97 °C
💧 Average Humidity: 63.96
🌧️ Rainy Days: 31
❄️ Snowy Days: 0
📉 Avg Pressure: 1008.56 millibars
🌫️ Avg Visibility: 12.19 km
🌬️ Avg Wind Speed: 9.64 km/h



🧑 You:  rainy days in june


🌧️ Number of rainy days in June: 330 days


🧑 You:  exit


👋 Goodbye!
